<a href="https://colab.research.google.com/github/vccheng2001/DeepLearning/blob/main/cnn_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 32 - 3 + 1 = 30
        self.conv = nn.Sequential (
            # 30*30*16
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            nn.ReLU(inplace=True),
            # 28*28*32
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.ReLU(inplace=True),
            # -> 26*26*64
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # 13 * 13 * 64
            nn.MaxPool2d(2,2),
            nn.Dropout2d(0.1),
            # 12 * 12 * 128
            nn.Conv2d(in_channels=64,out_channels=128, kernel_size=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            # -> 10*10*256
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3),
            nn.BatchNorm2d(256),

            nn.ReLU(inplace=True),
            # 5 * 5 * 256
            nn.MaxPool2d(2,2)
        )

        self.fc = nn.Sequential(
            nn.Linear(6400, 1024),
            nn.Dropout(0.05),
            nn.ReLU(inplace=True),
            nn.Linear(1024,512),
            nn.ReLU(inplace=True),
            nn.Linear(512,128),
            nn.Dropout(0.05),
            nn.Linear(128,10)
        )

    def forward(self, x):
        x = x.cuda()
        x = self.conv(x)
        x = x.view(x.shape[0],-1)
        x = self.fc(x)
        return x

PATH = './model.pth'

def main():
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    print(device)
    # load and transform dataset
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                            shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                            shuffle=False, num_workers=2)

    classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    net = CNN()
    net.to(device)
    # net.load_state_dict(torch.load(PATH))
   
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    num_epoch = 1
    for epoch in range(num_epoch):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 1000 == 999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

    torch.save(net.state_dict(), PATH)


    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

           
if __name__ == "__main__":
    main()



cuda
Files already downloaded and verified
Files already downloaded and verified
[1,  1000] loss: 1.070
[1,  2000] loss: 0.982
[1,  3000] loss: 0.921
[1,  4000] loss: 0.895
[1,  5000] loss: 0.871
[1,  6000] loss: 0.840
[1,  7000] loss: 0.814
[1,  8000] loss: 0.797
[1,  9000] loss: 0.765
[1, 10000] loss: 0.763
[1, 11000] loss: 0.751
[1, 12000] loss: 0.751
Finished Training
Accuracy of the network on the 10000 test images: 48 %
